In [22]:

from openai import OpenAI

client = OpenAI(api_key="sk-2b58c695f14649c1ab079e77dcae0deb",
                base_url="https://dashscope.aliyuncs.com/compatible-mode/v1")

In [23]:
import numpy as np
from numpy import dot
from numpy.linalg import norm

In [24]:
def cos_sim(a, b):
    '''
    余弦cos距离 -- 越大越相似
    :param a:
    :param b:
    :return:
    '''
    return dot(a, b) / (norm(a) * norm(b))


def l2(a, b):
    '''
    欧式距离-越小越相似
    :param a:
    :param b:
    :return:
    '''
    x = np.asarray(a) - np.asarray(b)
    return norm(x)

In [25]:
def get_embedding(texts, model="text-embedding-v1", dimensions=None):
    '''封装 openai 的 embedding 模型接口'''
    if model == "text-embedding-v1":
        dimensions = None
    if dimensions:
        data = client.embeddings.create(model=model, input=texts, dimensions=dimensions).data
    else:
        data = client.embeddings.create(model=model, input=texts).data
    return [i.embedding for i in data]

In [28]:
import pprint

query = ["这么近，那么远，周末来河北", "聚客 AI-用科技力量，构建智能未来"]
vec = get_embedding(query)[0]
print(f"Total dimension:{len(vec)}")
# print(f"first 10 elements:{vec[:10]}")
pprint.pprint(vec)


Total dimension:1536
[-0.9195741415023804,
 3.2635090351104736,
 -0.67822265625,
 2.873710870742798,
 -0.035378195345401764,
 1.4245753288269043,
 1.0740504264831543,
 -1.4042372703552246,
 2.4425315856933594,
 -0.8581209778785706,
 -0.8924597501754761,
 3.69492769241333,
 0.6750192642211914,
 -1.9036680459976196,
 0.15382662415504456,
 0.8413233757019043,
 -1.1260395050048828,
 3.8023922443389893,
 1.4888620376586914,
 -5.36575984954834,
 -2.9060797691345215,
 -1.772261142730713,
 -1.4105409383773804,
 -2.4302570819854736,
 -0.10710097849369049,
 -1.8359028100967407,
 -1.6955242156982422,
 1.3916010856628418,
 -1.891645908355713,
 -2.6834845542907715,
 1.452396273612976,
 0.3496916890144348,
 1.321475863456726,
 -1.3408350944519043,
 3.746931552886963,
 0.21651574969291687,
 1.9495766162872314,
 -3.3951988220214844,
 -4.19844913482666,
 -1.617658257484436,
 -2.606825351715088,
 3.5026776790618896,
 -2.3983635902404785,
 1.8720250129699707,
 0.34523335099220276,
 -0.5726466178894043,
 

In [29]:
query = "国际争端"
documents = [
    "联合国就苏丹达尔富尔地区大规模暴力事件发出警告",
    "士耳其、芬兰、瑞典与北约代表将维续就瑞典“入约”问题进行谈判",
    "日本岐阜市陆上自卫队射击场内发生枪击事件 3人受伤",
    "国家游试中心（水立方）：恢复游沐、嬉水乐园等水上项目运营",
    "我因首次在空间站开展舱外辐射生物学暴露实验"
]

query_vec = get_embedding([query])[0]
documents_vec = get_embedding(documents)

# pprint.pprint(query_vec)
# pprint.pprint(documents_vec)

print("query 与自己的余弦距离： {:.2f}".format(cos_sim(query_vec, query_vec)))
print("query 与 documents的余弦距离：")
for vec in documents_vec:
    print(cos_sim(query_vec, vec))

print("query 与自己的欧式距离：{:.2f}".format(l2(query_vec, query_vec)))
print("query 与 documents 的欧式距离：")
for vec in documents_vec:
    print(l2(query_vec, vec))


query 与自己的余弦距离： 1.00
query 与 documents的余弦距离：
0.33911047648730697
0.23329236002431183
0.07006357800727325
0.09675305155417296
0.1322302437882407
query 与自己的欧式距离：0.00
query 与 documents 的欧式距离：
109.51274483356154
110.7076814845259
128.00071081019382
128.12576638759367
126.5937711658174


In [14]:
import chromadb

client = chromadb.Client()


In [16]:
from chromadb.utils import embedding_functions

# 默认情况下，Chroma 使用 DefaultEmbeddingFunction, 是基于Sentence Transformers 的 MiniLm-L6-v2
default_ef = embedding_functions.DefaultEmbeddingFunction()

collection = client.create_collection(
    name="my_collecion2",
    configuration={
        # HNSW 索引算法，基于图的近似最近邻搜索算法 ANN
        "hnsw": {
            "space": "cosine",
            "ef_search": 100,
            "ef_construction": 100,
            "max_neighbors": 16,
            "num_threads": 4
        },
        # 指定向量模型
        "embedding_function": default_ef
    }
)

In [17]:
collection = client.get_collection(name="my_collecion")
print(pprint.pprint(collection.peek()))
print(collection.count())


{'data': None,
 'documents': [],
 'embeddings': array([], dtype=float64),
 'ids': [],
 'included': ['metadatas', 'documents', 'embeddings'],
 'metadatas': [],
 'uris': None}
None
0


In [19]:
collection.add(
    documents=["中国是ilen的故乡", "中国是ilen的故乡", "中国是ilen的故乡", "中国是ilen的故乡", "中国是ilen的故乡"]
    , metadatas=[{"source": "ilen"} for _ in range(5)]
    , ids=[str(i) for i in range(5)]
)

/Users/onepiecekevin/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:42<00:00, 1.94MiB/s]
